In [ ]:
!pip install pandas
!pip install openai

In [11]:
import pandas as pd
from openai import OpenAI

In [24]:
cap_table_data = {
    "Investor_Name": ["Founder 1", "Founder 2", "VC Firm A", "Angel Investor B", "VC Firm C", "Investor D"],
    "Holder_Type": ["Founder", "Founder", "Investor", "Investor", "Investor", "Investor"],
    "Shares_Held": [500000, 300000, 100000, 50000, 150000, 100000],
    "Investment_Type": ["Common Stock", "Common Stock", "Preferred Stock", "Preferred Stock", "Preferred Stock", "Common Stock"]
}

investor_quality_data = {
    'Investor_Name': ['Founder 1', 'Founder 2', 'VC Firm A', 'Angel Investor B', 'VC Firm C', 'Investor D'],
    'Reputation_Score': [8, 7, 9, 6, 8, 5],
    'Track_Record_Score': [7, 6, 9, 7, 8, 4],
    'Strategic_Value_Score': [8, 7, 10, 6, 9, 5]
}

cap_table = pd.DataFrame(cap_table_data)
investor_quality = pd.DataFrame(investor_quality_data)

cap_table['Ownership_Percentage'] = (cap_table['Shares_Held'] / cap_table['Shares_Held'].sum()) * 100
ownership_summary = cap_table.groupby('Holder_Type')['Ownership_Percentage'].sum()
investment_type_distribution = cap_table.groupby('Investment_Type')['Ownership_Percentage'].sum()
combined_data = cap_table.merge(investor_quality, on='Investor_Name')
combined_data['Weighted_Score'] = (combined_data['Reputation_Score'] * 0.4 +
                                   combined_data['Track_Record_Score'] * 0.3 +
                                   combined_data['Strategic_Value_Score'] * 0.3)

founder_ownership = ownership_summary['Founder']
investor_ownership = ownership_summary['Investor']
avg_reputation = combined_data['Reputation_Score'].mean()
avg_track_record = combined_data['Track_Record_Score'].mean()
avg_strategic_value = combined_data['Strategic_Value_Score'].mean()

# Display weights
(founder_ownership, investor_ownership, avg_reputation, avg_track_record, avg_strategic_value)

prompt_gippity = f"""
                Introduction: I'm analyzing a company's cap table for venture debt risk assessment.
                """
                
prompt_user = f"""
                Quantitative Summary: Founders hold {founder_ownership}% of the company, indicating strong control. Investors, including VC firms and an angel investor, hold {investor_ownership}%, with a significant portion in preferred stock. The majority of the ownership is in common stock.
                
                Qualitative Summary: The company's founders and key investors have been rated on reputation, track record, and strategic value. The average reputation score is {avg_reputation}/10, the average track record score is {avg_track_record}/10, and the average strategic value score is {avg_strategic_value}/10.
                
                Task: Between a score of 0% to 100%, estimate the risk percentage number before giving venture debt. Proceed with reasonings after.
                """
                

In [25]:
client = OpenAI()
completion = client.chat.completions.create(
  model="gpt-4-1106-preview",
  messages=[
    {
        "role": "system", 
        "content": f"""{prompt_gippity}"""
    },
    {   "role": "user", 
        "content": f"""{prompt_user}"""
    }
  ]
)


ChatCompletionMessage(content="To evaluate the risk associated with extending venture debt to this company, several factors need to be considered, both quantitative and qualitative. \n\nQuantitative factors include:\n\n1. **Ownership and Control**: The founders have a strong holding of 66.67%. This high percentage of founder ownership can be positive as it indicates that the founders are highly incentivized to ensure the success of the company. However, it also suggests that there might be less room for dilution to raise additional equity if needed. The 33.33% held by investors gives a reasonable balance of external oversight and influence.\n\n2. **Type of Securities Held**: The breakdown between common and preferred stock is crucial. Preferred stock typically has rights that are senior to common stock, and venture debt would be subordinated to preferred equity in the event of a liquidation. Therefore, knowing the terms of the preferred stock is important in assessing risks.\n\nQualita

In [26]:
# print(completion.choices[0].message)
print(completion.choices[0].message.content)

To evaluate the risk associated with extending venture debt to this company, several factors need to be considered, both quantitative and qualitative. 

Quantitative factors include:

1. **Ownership and Control**: The founders have a strong holding of 66.67%. This high percentage of founder ownership can be positive as it indicates that the founders are highly incentivized to ensure the success of the company. However, it also suggests that there might be less room for dilution to raise additional equity if needed. The 33.33% held by investors gives a reasonable balance of external oversight and influence.

2. **Type of Securities Held**: The breakdown between common and preferred stock is crucial. Preferred stock typically has rights that are senior to common stock, and venture debt would be subordinated to preferred equity in the event of a liquidation. Therefore, knowing the terms of the preferred stock is important in assessing risks.

Qualitative factors include:

1. **Reputation*